<a href="https://colab.research.google.com/github/ICPCfirst/DeepLearningHW2/blob/main/HW2(Generative).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

np.random.seed(0)
X_train_fpath = "/content/drive/MyDrive/data/X_train"
X_test_fpath = "/content/drive/MyDrive/data/X_test"
Y_train_fpath = "/content/drive/MyDrive/data/Y_train"
output_fpath = "./output_{}.csv"

In [ ]:
with open(X_train_fpath) as f:
  next(f)
  X_train = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
with open(X_test_fpath) as f:
  next(f)
  X_test = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
with open(Y_train_fpath) as f:
  next(f)
  Y_train = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
  

In [ ]:
def _normalize(X,train = True,specified_column = None,X_mean = None,X_std = None):
  if specified_column == None:
    specified_column = np.arange(X.shape[1])
  if train:
    X_mean = np.mean(X[:,specified_column],0).reshape(1,-1) # meean(0)==压缩行 ；reshape == 必须转化为一行n列 ？？？？？？？是不是重复了
    X_std = np.std(X[:,specified_column]).reshape(1,-1)
  
  X[:,specified_column] = (X[:,specified_column]-X_mean)/(X_std + 1e-8)# 进行归一化，中心化和标准处理化
  return X,X_mean,X_std

In [ ]:
def _train_dev_split(X,Y,dev_ratio = 0.25):
  train_size = int(len(X)*(1-dev_ratio))
  return X[:train_size],Y[:train_size],X[train_size:],Y[train_size:]

In [ ]:
X_train,X_mean,X_std = _normalize(X_train,train = True)
X_test,_,_ = _normalize(X_test,train = False,specified_column=None,X_mean = X_mean,X_std = X_std)

dev_ratio = 0.1
X_train,Y_train,X_dev,Y_dev = _train_dev_split(X_train,Y_train,dev_ratio = dev_ratio) #自己根据已给数据分开来使程序准确性更高

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]